In [8]:
!pip install pyspark==3.2.1
!pip install katonic
!pip install boto3

from pyspark.sql.functions import col, to_timestamp
from pyspark.sql.types import *
from katonic.connectors.python.s3 import AWSS3Connector        #using Katonic SDK
 
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 161.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 62.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 178.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 253.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 184.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.5/145.5 kB 257.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 284.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 230.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('demo').getOrCreate()

In [4]:
from minio import Minio
from sqlalchemy import create_engine

ACCESS_KEY = "DNPD2SAYLBELJ423HCNU"
SECRET_KEY = "zF7F6W93HS8vt+JKen4U17+zhcHiwH47AMuO3ap0"
PUBLIC_BUCKET = "shared-storage"

client = Minio(
    endpoint="minio-server.default.svc.cluster.local:9000",
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    secure=False,
)

In [6]:
data_path = "s3_access_creds.json"

client.fget_object(
            'private-storage-f6047024',
            data_path,
            "s3_access_creds.json",
        )

In [11]:
import json

with open('s3_access_creds.json') as f:
    config = json.load(f)

In [12]:
df = AWSS3Connector(
    aws_access_key=config["AWS_ACCESS_KEY"],
    aws_secret_key=config["AWS_SECRET_KEY"],
    region_name=config["REGION_NAME"],
    bucket_name=config["BUCKET_NAME"],
    object_name=config["OBJECT_NAME"],
    output="local",
    file_name="attribution_data.csv",
)

In [13]:
df.get_data()

Connection instance to aws s3 established Successfully.
File saved to your 'local'file system with name 's3_attribution_data.csv_2023_04_18_15_30_25.csv' Successfully.


In [15]:
data_path = "/multi_touch_attribution/s3_attribution_data.csv_2023_04_18_15_30_25.csv"

client.fput_object(
            PUBLIC_BUCKET,
            data_path,
            "s3_attribution_data.csv_2023_04_18_15_30_25.csv",
        )

In [16]:
data_path = 'attribution_data.csv'

data = spark.read.format('csv').option('header','true').load('s3_attribution_data.csv_2023_04_18_15_30_25.csv')
data.show()

+--------------------+-------------------+-----------+--------------------+----------+
|                 uid|               time|interaction|             channel|conversion|
+--------------------+-------------------+-----------+--------------------+----------+
|ec66fa36ab5e42558...|2020-05-22 04:31:17| impression|Search Engine Mar...|         0|
|ec66fa36ab5e42558...|2020-06-07 20:05:15| impression|          Affiliates|         0|
|ec66fa36ab5e42558...|2020-06-09 14:14:47| impression|          Affiliates|         0|
|ec66fa36ab5e42558...|2020-06-09 16:17:36| impression|Search Engine Mar...|         0|
|ba0c1d0c041a4a539...|2020-06-05 01:18:50| impression|Search Engine Mar...|         0|
|ba0c1d0c041a4a539...|2020-06-06 03:06:05| impression|Search Engine Mar...|         0|
|ba0c1d0c041a4a539...|2020-06-09 12:06:42| impression|          Affiliates|         0|
|ba0c1d0c041a4a539...|2020-06-10 03:33:17| impression|Search Engine Mar...|         0|
|ba0c1d0c041a4a539...|2020-06-10 04:15:42| 